In [1]:
from creating_arrays import CHART_TIME_ARRAY,MONTH_ARRAY,SYMBOL_ARRAY;
from constants import BINANCE_URL, download_dir, output_dir,dtypes;
from pathlib import Path;
import os;
import re;
import shutil;
from selenium import webdriver;
import talib;
import datetime;
from selenium.webdriver.chrome.options import Options;
import glob;
import zipfile;
import pandas as pd;
from pandas.api.types import is_numeric_dtype;
import inspect
import talib
import time
import numpy as np

['1000BTTCUSDT', '1000LUNCBUSD', '1000LUNCUSDT', '1000SHIBBUSD', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ADABUSD', 'ADAUSDT', 'AKROUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'AMBBUSD', 'ANCBUSD', 'ANCUSDT', 'ANKRUSDT', 'ANTUSDT', 'APEBUSD', 'APEUSDT', 'API3USDT', 'APTBUSD', 'APTUSDT', 'ARPAUSDT', 'ARUSDT', 'ATAUSDT', 'ATOMUSDT', 'AUCTIONBUSD', 'AUDIOUSDT', 'AVAXBUSD', 'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BLUEBIRDUSDT', 'BLZUSDT', 'BNBBUSD', 'BNBUSDT', 'BNXUSDT', 'BTCBUSD', 'BTCBUSD_210129', 'BTCBUSD_210226', 'BTCDOMUSDT', 'BTCSTUSDT', 'BTCUSDT', 'BTCUSDT_210326', 'BTCUSDT_210625', 'BTCUSDT_210924', 'BTCUSDT_211231', 'BTCUSDT_220325', 'BTCUSDT_220624', 'BTCUSDT_220930', 'BTCUSDT_221230', 'BTCUSDT_230331', 'BTSUSDT', 'BTTUSDT', 'BZRXUSDT', 'C98USDT', 'CELOUSDT', 'CELRUSDT', 'CHRUSDT', 'CHZUSDT', 'COMPUSDT', 'COTIUSDT', 'CRVUSDT', 'CTKUSDT', 'CTSIUSDT', 'CVCUSDT', 'CVXBUSD', 'CVXUSDT', 'DARUSDT', 'DASHUSDT', 'DEFIUS

In [2]:
# Get the current date
now = datetime.datetime.now()

# Format the date as a string in the desired format
date_string = now.strftime("%Y-%m")

# Print the date string
print(date_string)

idx = MONTH_ARRAY.index(date_string)
MONTH_ARRAY = MONTH_ARRAY[:idx+1]


2023-01


## Set up the arrays of symbols, chart times, and months

In [3]:
print(SYMBOL_ARRAY.index('BTCUSDT'))

49


In [4]:
# Set up the arrays of symbols, chart times, and months
symbols = [SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')],SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHUSDT')],SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHBUSD')],SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCBUSD')]]
# symbols = [SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')]]

# chart_times = CHART_TIME_ARRAY
chart_times = [CHART_TIME_ARRAY[12]]
months = MONTH_ARRAY

## Download the data

In [5]:
try:
    # Set up the ChromeOptions instance
    options = Options()

    # Set the download directory in the prefs dictionary
    prefs = {"download.default_directory": download_dir}
    options.add_experimental_option("prefs", prefs)

    # Set up the webdriver
    driver = webdriver.Chrome(options=options)

    # Open the webpage
    driver.get(BINANCE_URL)



    # Iterate over the symbols, chart times, and months
    for symbol in symbols:
        for chart_time in chart_times:
            for month in months:
                # Construct the link
                link = f"{BINANCE_URL}{symbol}/{chart_time}/{symbol}-{chart_time}-{month}.zip"
    #             print(link)
                # Open the link in a new tab
                driver.execute_script(f"window.open('{link}', '_blank');")

#     # Wait for the downloads to finish
#     driver.implicitly_wait(60)

    # Close the webdriver
    driver.close()
except:
    pass



## Move to separate folders. Caution: run this only after the downloads are finished.

In [6]:
for symbol in symbols:
    for chart_time in chart_times:
        # Create the new folder path
        new_folder_path = Path(download_dir) / f"{symbol}-{chart_time}"
        new_folder_path.mkdir(parents=True, exist_ok=True)
        # match pattern of the file
        pattern = re.compile(f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")
        # list all files in the download directory
        files = os.listdir(download_dir)
        for file in files:
            if pattern.match(file):
                src = Path(download_dir) / file
                dst = new_folder_path / file
                src.rename(dst)


## Unzip the Zip files and concatenate the CSVs

### Move the monthly zips and csvs to a new folder.
#### Earlier, I wanted to delete them completely. But then I realized binance will not give more than 24 months of data at a time so now that we're in 2023, we wont get 2020 data. So I have decided not to delete those. (okay im not sure about this but i still dont want to risk)

In [7]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Set up an empty list for the data frames
        df_list = []

        # Compile the regular expression pattern
        pattern = re.compile(f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

        # Create the new folder path for ZIP files
        new_zip_folder_path = os.path.join(download_dir, f"{symbol}-{chart_time}")

        # Create the new folder path for CSV files
        new_csv_folder_path = os.path.join(output_dir, f"{symbol}-{chart_time}")

        # Iterate over the files in the new zip folder
        for file in os.listdir(new_zip_folder_path):
            # Check if the file matches the pattern
            if pattern.match(file):
                # Construct the file path
                file_path = os.path.join(new_zip_folder_path, file)

                # Extract the ZIP file
                with zipfile.ZipFile(file_path, "r") as zip_ref:
                    zip_ref.extractall(new_csv_folder_path)

                # Construct the CSV file path
                csv_file_path = os.path.join(new_csv_folder_path, f"{symbol}-{chart_time}{file[-12:-4]}.csv")

                # Read the CSV file into a data frame, ignoring the headers
                df = pd.read_csv(csv_file_path, header=None)

                # Remove the first row (which contains the header)
                df = df.iloc[1:]

                # Add it to the list
                df_list.append(df)

        # Concatenate the data frames in the list
        df_final = pd.concat(df_list, ignore_index=True)

        # Read the headers from the first CSV file
        headers = pd.read_csv(csv_file_path, nrows=1).columns

        # Set the headers as the column names of the final dataframe
        df_final.columns = headers
        
        # Convert 'open_time' and 'close_time' columns to datetime
        df_final['open_time'] = pd.to_datetime(df_final['open_time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
        df_final['close_time'] = pd.to_datetime(df_final['close_time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
        
        # Delete the 'ignore' column
        df_final = df_final.drop(['ignore'], axis=1)
        
        # Add a new column called 'entry' that will take previous close
        df_final['entry'] = df_final['close'].shift(1)
        # Set the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = os.path.join(new_csv_folder_path, file_name)

        # Write the data frame to the Excel file
        df_final.to_csv(file_path, index=False)


## Master cell to add different indicators to the dataframe and save it


In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        # Add the RSI as a new column to the dataframe
        df['RSI'] = talib.RSI(df['close'], timeperiod=8)
        
        # Add the MACD as new columns to the dataframe
        df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)


In [ ]:
indicators_master_list = talib.get_functions()
indicators_list =[indicators_master_list[40],indicators_master_list[54]]
print(indicators_list)

## Each candle is an entry. Below is the code to know if shorts wouldve won or longs wouldve won

In [ ]:
win_perc = 0.5
loss_perc = 0.4

In [ ]:
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        df["if_short"] = 0
        df["if_long"] = 0
        df["long_target"] = np.nan
        df["short_target"] = np.nan
        df["long_stop_loss"] = np.nan
        df["short_stop_loss"] = np.nan
        df["shorts_win_after"] = np.nan
        df["longs_win_after"] = np.nan
        
        for i in range(len(df)):
            if df["entry"][i]:
                long_target = df["entry"][i]*(1+win_perc/100)
                short_target = df["entry"][i]*(1-win_perc/100)
                long_stop_loss = df["entry"][i]*(1-loss_perc/100)
                short_stop_loss = df["entry"][i]*(1+loss_perc/100)
                df.loc[i,'long_target'] = long_target
                df.loc[i,'long_stop_loss'] = long_stop_loss
                for j in range(i,len(df)):
                    if df["high"][j] >= long_target:
                        df.loc[i,'if_long'] = 1
                        df.loc[i,'longs_win_after'] = j-i
                        break
                    if df["low"][j] <= long_stop_loss:
                        df.loc[i,'if_long'] = -1
                        break
                df.loc[i,'short_target'] = short_target
                df.loc[i,'short_stop_loss'] = short_stop_loss
                for j in range(i,len(df)):
                    if df["low"][j] <= short_target:
                        df.loc[i,'if_short'] = 1
                        df.loc[i,'shorts_win_after'] = j-i
                        break
                    if df["high"][j] >= short_stop_loss:
                        df.loc[i,'if_short'] = -1
                        break
        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)


In [ ]:
# Get the list of available functions in TA-LIB
indicators_master_list = talib.get_functions()
indicators_list =[indicators_master_list[40],indicators_master_list[54]]

# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)

        # Iterate over the indicators in the indicator_list
        for indicator in indicators_list:
            # Get the function's input signature
            sig = inspect.signature(indicator)
            print(sig)
            # Check if the function requires close prices
            if 'close' in [p.name for p in sig.parameters.values()]:
                try:
                    # Add the indicator as a new column to the dataframe
                    df[indicator] = func(df['close'])
                except Exception as e:
                    print(f"{indicator} failed with error: {e}")
                    pass
        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)


### Troubleshooting utilities. Can be put in another notebook 


In [ ]:
column_names = pd.read_csv(csv_file_path, nrows=0).columns
print(column_names)

In [ ]:
# Set the chart time and symbol
chart_time = "5m"
symbol = "BTCBUSD"

# Set the file name
file_name = f"{symbol}-{chart_time}.csv"

# Construct the file path
file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

# Read the CSV file into a dataframe
df = pd.read_csv(file_path)

# Iterate over the rows of the dataframe
for i, row in df.iterrows():
    try:
        # Convert the 'open' column to float
        open_value = float(row['open'])
    except ValueError:
        # Print the value that caused the error
        print(row['1'])


## Extras

#### Chat gpt suggested this when i asked it for this:
absolutely fantastic, currently i have rsi and macd. can you make a model that will look into the wins and losses and the values of rsi and macd for them and find the tightest range of the combination of macd and rsi that has a winrate of 71%

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Load the data into a dataframe
df = pd.read_csv("data.csv")

# Define the features and target variables
X = df[["RSI", "MACD"]]
y = df["if_long"]

# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# Fit the model to the data
clf.fit(X, y)

# Get the feature importances
importances = clf.feature_importances_

# Print the feature importances
print("Feature importances:", importances)

# Get the indices of the top N features
N = 2
indices = np.argsort(importances)[::-1][:N]

# Print the top N features
print("Top {N} features:", [X.columns[i] for i in indices])
